#### 자연어 데이터셋 전처리 
<hr>

- Korpora 패키지 활용
- 데이터셋 : 한국어 혐오 데이터셋


In [1]:
# 패키지 설치 확인
import Korpora
import pandas as pd

Korpora.__version__

'0.2.0'

#### 데이터 다운로드

In [2]:
from Korpora import KoreanPetitionsKorpus, KoreanHateSpeechKorpus, Korpora

# 데이터셋의 클래스, 인스턴스 생성시 자동 다운로드
corpus = KoreanHateSpeechKorpus()



    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Authors :
        - Jihyung Moon* (inmoonlight@github)
        - Won Ik Cho* (warnikchow@github)
        - Junbum Lee (beomi@github)
        * equal contribution
    Repository : https://github.com/kocohub/korean-hate-speech
    References :
        - Moon, J., Cho, W. I., & Lee, J. (2020). BEEP! Korean Corpus of Online News
          Comments for Toxic Speech Detection. arXiv preprint arXiv:2005.12503.

    We provide the first human-annotated Korean corpus for toxic speech detection and the large unlabeled corpus.
    The data is comments from the Korean entertainment news aggregation platform.

    # License
    Creative Commons Attribution-ShareAlike 4.0 International License.
    Visit here for detail : https://creativec

In [3]:
hateDataset = corpus.train
hateDatasettest = corpus.test
hateDatasetdev = corpus.dev

In [4]:
[x for x in dir(hateDataset) if x.find("__") < 0]

['biases',
 'gender_biases',
 'get_all_texts',
 'hates',
 'name',
 'texts',
 'titles']

#### (2) 데이터 전처리
<hr>

- 주제에 따른 데이터 피쳐 & 라벨 선택
- 형태소 분석기 결정
- 

In [5]:
df = pd.DataFrame((zip(hateDataset.texts, hateDataset.gender_biases, hateDataset.titles, hateDataset.hates, hateDataset.biases)))
df.columns = ["text", "gender_biases", "titles", "hates", 'biases']
df[df['hates'] == 'hate']


,text,gender_biases,titles,hates,biases
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,"""밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]""",hate,others
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,"""[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)""",hate,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”,hate,gender
6,"100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, ...",True,"""배우 마크 월버그, 성차별 논란에 출연료 16억원 전액 기부""",hate,gender
19,180이하 호빗 한남들은 결혼 하지마셈 ㅋ 돈없으면 연애도 하지마셈 ㅋ 니들 호빗 ...,True,"""'라스' 남주혁 """"중학교때 이미 184cm, 2년만에 키 30cm 폭풍 성장""""""",hate,gender
...,...,...,...,...,...
7864,흑발미녀???장난똥때리냐?????마녀란 말도 아깝다!!!,False,"""낸시랭, 왕진진과 결혼 후 근황..흑발 미녀의 청순""",hate,others
7879,히잌! 기저귀 찬 년들이 군대 군대 주댕이 털고 자빠졌노ㅋ( ° ͜ʖ͡°)╭∩╮,True,"""""""여성 혐오NO""""..산이, '이수역 폭행' 영상 게재→신곡 '페미니스트' 발표...",hate,gender
7880,히트곡이 없는데 추억팔이가되는 놀라운 조선식 방송국연예매니징ㅋㅋ 기획사 소속사 걍 ...,False,"""""""이 정도면 신드롬""""..'연예인들의 연예인' 양준일, 이지혜→김이나→신현준도 ...",hate,others
7881,"히트작이래봐야 미사, 발리 두개 정도가 다고 연기력도 그저 그런 수준으로 배우로서 ...",False,[종합] 소지섭 측 “한남 더 힐 매입·이사…♥조은정과 결혼·신혼집? NO”,hate,none


In [6]:
import konlpy.tag as tag

mecab = tag.Mecab()

In [7]:
df.biases.unique()

array(['others', 'none', 'gender'], dtype=object)

In [8]:
df[df['biases'] == 'gender']['text'].to_csv('traintext.txt', index=False, encoding='utf-8')

In [9]:
from sentencepiece import SentencePieceTrainer

SentencePieceTrainer.Train(
    '--input=traintext.txt\
    --model_prefix=hate_bpe\
    --vocab_size=10000\
    --model_type=bpe'
)

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=traintext.txt    --model_prefix=hate_bpe    --vocab_size=10000    --model_type=bpe
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: traintext.txt
  input_format: 
  model_prefix: hate_bpe
  model_type: BPE
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2

In [10]:
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load('hate_bpe.model')

sentence=  '안녕하세요 오늘도 아침입니다.'

token_sentence = tokenizer.encode_as_pieces(sentence)
token_sentence

['▁안', '녕', '하세요', '▁오늘', '도', '▁아침', '입니다', '.']

In [11]:
encodeID = tokenizer.encode_as_ids(sentence)
encodeID

[17, 9673, 1278, 1543, 8846, 3135, 882, 8833]

In [12]:
tokenizer.decode_ids(encodeID), tokenizer.DecodePieces(token_sentence)

('안녕하세요 오늘도 아침입니다.', '안녕하세요 오늘도 아침입니다.')

In [13]:
vocab = {idx: tokenizer.IdToPiece(idx) for idx in range(tokenizer.get_piece_size())}
list(vocab.items()), len(vocab.keys())

([(0, '<unk>'),
  (1, '<s>'),
  (2, '</s>'),
  (3, 'ᄏᄏ'),
  (4, '..'),
  (5, '▁여'),
  (6, '▁이'),
  (7, '▁여자'),
  (8, '▁아'),
  (9, '▁남'),
  (10, '▁그'),
  (11, '▁남자'),
  (12, '▁한'),
  (13, '▁하'),
  (14, 'ᄏᄏᄏᄏ'),
  (15, '...'),
  (16, '▁사'),
  (17, '▁안'),
  (18, '▁나'),
  (19, '들이'),
  (20, '하는'),
  (21, '▁다'),
  (22, '▁보'),
  (23, '▁어'),
  (24, '▁잘'),
  (25, '▁성'),
  (26, '하고'),
  (27, '▁ᄏᄏ'),
  (28, '▁가'),
  (29, '▁저'),
  (30, '▁있'),
  (31, '여자'),
  (32, '▁결'),
  (33, '▁지'),
  (34, '들은'),
  (35, '▁좋'),
  (36, '는데'),
  (37, '▁못'),
  (38, '▁자'),
  (39, '▁진'),
  (40, '▁연'),
  (41, '▁왜'),
  (42, '는거'),
  (43, '▁결혼'),
  (44, '!!'),
  (45, '▁아니'),
  (46, '▁기'),
  (47, '다고'),
  (48, '▁생'),
  (49, '▁없'),
  (50, '▁"'),
  (51, '▁김'),
  (52, '??'),
  (53, '~~'),
  (54, '▁살'),
  (55, '▁인'),
  (56, '▁애'),
  (57, '▁대'),
  (58, '으로'),
  (59, '▁만'),
  (60, '으면'),
  (61, '▁같'),
  (62, '▁마'),
  (63, '▁너'),
  (64, '▁정'),
  (65, '▁한남'),
  (66, '▁조'),
  (67, '▁미'),
  (68, '▁무'),
  (69, '▁전'),
  (70, '▁많'),
 

In [14]:
with open('traintext.txt', 'r') as f:
    result = f.read()
    test = mecab.nouns(result)

test

['사람',
 '얼굴',
 '손톱',
 '것',
 '인격',
 '살해',
 '동영상',
 '몰카',
 '걸리',
 '안들',
 '생각',
 '년',
 '안',
 '남녀',
 '간',
 '전쟁',
 '번',
 '후',
 '일부다처제',
 '여성',
 '정치',
 '참여',
 '금지',
 '여성',
 '투표',
 '삭제',
 '세계',
 '공통',
 '문화',
 '자리',
 '듯',
 '암탉',
 '이하',
 '남',
 '결혼',
 '셈',
 '돈',
 '연애',
 '셈',
 '니',
 '유전자',
 '고통',
 '후손',
 '죄',
 '대',
 '골',
 '여자',
 '기사',
 '건가',
 '여자',
 '투표',
 '엠',
 '넷',
 '사전',
 '투표',
 '계집',
 '대',
 '대',
 '남자',
 '비율',
 '보소',
 '짓',
 '거',
 '년',
 '전',
 '한참',
 '팔',
 '팔',
 '때',
 '오달수',
 '이',
 '십',
 '대',
 '초반',
 '때',
 '반',
 '콩',
 '거',
 '세상',
 '성범죄자',
 '인간',
 '년',
 '전',
 '오달수',
 '혈기',
 '왕',
 '때',
 '수',
 '듯',
 '년',
 '일',
 '말',
 '사람',
 '참',
 '충실',
 '대녀',
 '꿀',
 '차',
 '남자',
 '집',
 '남자',
 '탓',
 '일',
 '대',
 '아줌마',
 '남',
 '집',
 '대',
 '아줌마',
 '대',
 '여자',
 '군대',
 '군대',
 '이야기',
 '관심',
 '이야기',
 '탑',
 '대',
 '여자',
 '비율',
 '거',
 '고소영',
 '이유',
 '여자',
 '진심',
 '한마디',
 '대',
 '여자',
 '진심',
 '여자',
 '대',
 '여자',
 '한혜진',
 '빙의',
 '플',
 '거',
 '본인',
 '인생',
 '여자',
 '질투',
 '살',
 '시집',
 '뭔가',
 '문제',
 '여자',
 '건모',
 '상대',
 '필요',
 '대',
 '할

In [15]:
# !pip install tokenizersd

In [16]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.normalizers import Lowercase, Sequence, NFD
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.decoders import WordPiece as WordPieceDecoder

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Sequence([NFD(), Lowercase()])
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train(['./traintext.txt'])
tokenizer.save('./tokenizer.json')

In [17]:
tokenizer = Tokenizer.from_file('./tokenizer.json')
tokenizer.decoder = WordPieceDecoder()


In [18]:
tokenizer.encode('안녕하세요 오늘도 아침입니다.').tokens

['안', '##녕', '##하세요', '오늘도', '아침', '##입니다', '.']